In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import os
import zipfile
from io import BytesIO

# Путь к папке с ZIP-архивами
folder_path = 'historical_tiobe_pypl'

# Создаем объединенный DataFrame
full_data = pd.DataFrame()

# Обрабатываем каждый ZIP-файл
for file in os.listdir(folder_path):
    if file.endswith('.zip'):
        # Извлекаем год из имени файла
        try:
            year = int(file.split('_')[1].split('.')[0])
        except:
            print(f"Не удалось извлечь год из имени файла: {file}")
            continue
            
        zip_path = os.path.join(folder_path, file)
        
        # Открываем ZIP-архив
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Ищем CSV-файл внутри архива
            csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
            
            if not csv_files:
                print(f"В архиве {file} не найден CSV-файл")
                continue
                
            # Берем первый найденный CSV-файл
            csv_name = csv_files[0]
            
            # Читаем CSV-файл прямо из архива
            with zip_ref.open(csv_name) as csv_file:
                # Используем BytesIO для pandas
                df = pd.read_csv(BytesIO(csv_file.read()))
                
                # Добавляем колонку с годом
                df['year'] = year
                
                # Объединяем с основным DataFrame
                full_data = pd.concat([full_data, df], ignore_index=True)

# Функция для обработки колонок с перечисленными значениями
def process_column(data, column_name):
    # Разделяем значения по точке с запятой
    result = data[column_name].str.split(';', expand=True)
    
    # Преобразуем в длинный формат
    result = result.stack().reset_index(level=1, drop=True).reset_index(name='item')
    result['item'] = result['item'].str.strip()
    
    # Удаляем пустые значения
    result = result[result['item'] != '']
    result = result.dropna(subset=['item'])
    
    return result['item'].value_counts(normalize=True).mul(100)

# 1. Топ языков программирования (2024)
data_2024 = full_data[full_data['year'] == 2024]
lang_worked = process_column(data_2024, 'LanguageHaveWorkedWith')
lang_admired = process_column(data_2024, 'LanguageAdmired')

plt.figure(figsize=(14, 8))
plt.subplot(1, 2, 1)
lang_worked.head(10).plot(kind='barh', color='skyblue')
plt.title('Топ языков по использованию (2024)')
plt.xlabel('Доля респондентов (%)')

plt.subplot(1, 2, 2)
lang_admired.head(10).plot(kind='barh', color='salmon')
plt.title('Топ языков по предпочтениям (2024)')
plt.xlabel('Доля респондентов (%)')

plt.tight_layout()
plt.show()

# 2. Динамика популярности языков (2019-2024)
def get_top_languages_by_year(year):
    year_data = full_data[full_data['year'] == year]
    lang_share = process_column(year_data, 'LanguageHaveWorkedWith')
    return lang_share.head(5)

# Создаем DataFrame для динамики
trend_data = pd.DataFrame()
years = sorted(full_data['year'].unique())

for year in years:
    top_langs = get_top_languages_by_year(year)
    for lang, share in top_langs.items():
        trend_data = pd.concat([trend_data, pd.DataFrame({
            'year': [year],
            'language': [lang],
            'share': [share]
        })], ignore_index=True)

plt.figure(figsize=(14, 8))
sns.lineplot(data=trend_data, x='year', y='share', hue='language', 
             marker='o', linewidth=2.5)
plt.title('Динамика популярности топ-5 языков (2019-2024)')
plt.xlabel('Год')
plt.ylabel('Доля респондентов (%)')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 3. Образование и опыт
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Уровень образования (2024)
education = data_2024['EdLevel'].value_counts(normalize=True).mul(100)
education.plot.pie(autopct='%1.1f%%', ax=ax[0], startangle=90)
ax[0].set_title('Уровень образования (2024)')
ax[0].set_ylabel('')

# Опыт работы (2024)
experience = data_2024['Age'].value_counts().sort_index()
experience.plot(kind='bar', ax=ax[1], color='teal')
ax[1].set_title('Опыт работы (2024)')
ax[1].set_xlabel('Годы опыта')
ax[1].set_ylabel('Количество респондентов')

plt.tight_layout()
plt.show()

# 4. Рабочие практики (2024)
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Формат работы
remote = data_2024['RemoteWork'].value_counts(normalize=True).mul(100)
remote.plot.bar(ax=ax[0], color='purple')
ax[0].set_title('Формат работы (2024)')
ax[0].set_ylabel('Доля респондентов (%)')

# Активность в кодинге
coding = data_2024['Check CodingActivities'].value_counts(normalize=True).mul(100)
coding.plot.bar(ax=ax[1], color='orange')
ax[1].set_title('Активность в программировании (2024)')
ax[1].set_ylabel('Доля респондентов (%)')

plt.tight_layout()
plt.show()

# 5. Обучение программированию (2024)
fig = px.sunburst(
    data_2024,
    path=['LearnCode', 'LearnCodeOnline'],
    title='Методы обучения программированию (2024)'
)
fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=30, l=0, r=0, b=0))
fig.show()

# Ключевые выводы
print("\nКлючевые выводы:")
print(f"1. Самый используемый язык в 2024: {lang_worked.index[0]} ({lang_worked.iloc[0]:.1f}%)")
print(f"2. Самый желаемый язык в 2024: {lang_admired.index[0]} ({lang_admired.iloc[0]:.1f}%)")
print(f"3. Основной формат работы: {remote.index[0]} ({remote.iloc[0]:.1f}%)")
print(f"4. Преобладающий уровень образования: {education.index[0]} ({education.iloc[0]:.1f}%)")
print(f"5. Наиболее активные в программировании: {coding.index[0]} ({coding.iloc[0]:.1f}%)")

# Проверка данных
print("\nСтатистика данных:")
print(f"- Всего записей: {len(full_data)}")
print(f"- Годы данных: {sorted(full_data['year'].unique())}")
print(f"- Колонки: {list(full_data.columns)}")